## 七月在线机器学习实训营三期第七周(机器学习-模型调优与融合)考试
#### 考试说明:
- 起止时间：请同学在2018年3月23日至3月25日期间完成，最晚提交时间本周日（3月25日24时之前）结束，<b>逾期不接受补考,该考试分数计入平时成绩</b>
- 考试方式：请同学<font color=red><b>拷贝</b></font>该试卷至自己姓名的目录后，将文件更名为同学姓名拼音-exam7后，进行作答。例如wangwei-exam7
- 提交格式：请同学新建自己姓名全拼的文件夹，将该试卷，数据文件，zip文件等相关考试文件，放置此目录下。将该目录<b>移动</b>至/0.Teacher/Exam/7/目录下
- 注意事项：为确保同学们真正了解自身对本周课程的掌握程度，<font color=red><b>请勿翻阅，移动，更改</b></font>其它同学试卷。如发现按0分处理
- 请同学在下方同学姓名处填写自己的姓名，批改人和最终得分不用填写

- 同学姓名:王玺 
- 批改人： David
- 最终得分:100

<center><h1>####答卷开始####</h1></center>
<hr>

## 简答题(共6题，1，2题每题5分，后4题每题10分，共计50分)

- note :50

#### 1. 机器学习任务中，通常会将给定数据切分为训练集，验证集，测试集，请回答这三类数据集各自的用途。

- note: 一般数据科学比赛时的测试集是不带标签，防止做BI，但本地做项目时候，测试集本就是数据集中的一部分哈。

训练集(train set) 是带标签的，用来拟合，训练模型，确定模型参数的(如LR中的w等),从而用这部分数据来建立模型。  

验证集（validation set）也是带标签的，用来做模型选择（model selection），即做模型的最终优化及确定的；常用的方法是是留少部分做测试集。然后对其余N个样本采用K折交叉验证法。就是将样本打乱，然后均匀分成K份，轮流选择其中K－1份训练，剩余的一份做验证，计算预测误差平方和，最后把K次的预测误差平方和再做平均作为选择最优模型结构的依据。K取N，就是留一法（leave one out）。  

测试集（test set）也是带标签的，是为了测试已经训练好的模型的推广能力。当然，test set这并不能保证模型的正确性，他只是展示了相似的数据用此模型会得出的结果。  

#### 2.如何将数据转换成xgboost内置用法的libsvm数据格式，以及该格式数据是如何解读的?

libsvm格式十分适合存储稀疏的数据集。最左边的一列是标签的值。其余位置存放的是有值的特征的下标和它对应的值。
label index1:value1 index2:value2……
下面是将数据转换成libsvm的方法

In [1]:
from sklearn.datasets import load_iris
import xgboost as xgb
from sklearn.model_selection import train_test_split
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

#### 3. 对于自动特征选择，通常有哪三类方法？试写出每类方式的思想，以及在sklearn中的代码实现（面试题）

1.过滤式(filter)            
Filter特征选择方法是一种启发式方法，其基本思想是：制定一个准则，用来衡量每个特征/属性，对目标属性的重要性程度，以此来对所有特征/属性进行排序，或者进行优选操作。常用的衡量准则有假设检验的p值、相关系数、互信息、信息增益等。

1）方差选择法：使用方差选择法，先要计算各个特征的方差，然后根据阈值，选择方差大于阈值的特征。使用feature_selection库的VarianceThreshold类来选择特征的代码如下：    
from sklearn.feature_selection import VarianceThreshold       
VarianceThreshold(threshold=3).fit_transform(iris.data)     

2） 相关系数法：使用相关系数法，先要计算各个特征对目标值的相关系数以及相关系数的P值。用feature_selection库的SelectKBest类结合相关系数来选择特征的代码如下：     
from sklearn.feature_selection import SelectKBest       
from scipy.stats import pearsonr      
SelectKBest(lambda X, Y: array(map(lambda x:pearsonr(x, Y), X.T)).T, k=2).fit_transform(iris.data, iris.target)   

3）卡方检验：经典的卡方检验是检验定性自变量对定性因变量的相关性。假设自变量有N种取值，因变量有M种取值，考虑自变量等于i且因变量等于j的样本频数的观察值与期望的差距，构建统计量不难发现，这个统计量的含义简而言之就是自变量对因变量的相关性。用feature_selection库的SelectKBest类结合卡方检验来选择特征的代码如下：       
from sklearn.feature_selection import SelectKBest       
from sklearn.feature_selection import chi2      
SelectKBest(chi2, k=2).fit_transform(iris.data, iris.target)

4）互信息法：经典的互信息也是评价定性自变量对定性因变量的相关性的，为了处理定量数据，最大信息系数法被提出，使用feature_selection库的SelectKBest类结合最大信息系数法来选择特征的代码如下：       
from sklearn.feature_selection import SelectKBest      
from minepy import MINE      
 def mic(x, y):    
     m = MINE()     
     m.compute_score(x, y)     
     return (m.mic(), 0.5)      
SelectKBest(lambda X, Y: array(map(lambda x:mic(x, Y), X.T)).T, k=2).fit_transform        

2.包裹式（wrapper）       
其主要思想是：将子集的选择看作是一个搜索寻优问题，生成不同的组合，对组合进行评价，再与其他的组合进行比较。这样就将子集的选择看作是一个是一个优化问题，这里有很多的优化算法可以解决，尤其是一些启发式的优化算法，如GA，PSO，DE，ABC等。  

1）使用feature_selection库的RFE类来选择特征的代码如下：       
from sklearn.feature_selection import RFE        
from sklearn.linear_model import LogisticRegression                   
RFE(estimator=LogisticRegression(), n_features_to_select=2).fit_transform(iris.data, iris.target)              

3.嵌入式（embedding）    
嵌入式特征选择将特征选择过程和机器训练过程融合为一体。两者在同一优化过程中完成，即在学习器训练过程中自动进行了特征选择。。比如lasso回归，岭回归，弹性网络（Elastic Net）等。

1）基于惩罚项的特征选择法:使用带惩罚项的基模型，除了筛选出特征外，同时也进行了降维。使用feature_selection库的SelectFromModel类结合带L1惩罚项的逻辑回归模型，来选择特征的代码如下：     
from sklearn.feature_selection import SelectFromModel             
from sklearn.linear_model import LogisticRegression                       
SelectFromModel(LogisticRegression(penalty="l1", C=0.1)).fit_transform(iris.data, iris.target)    

2）使用feature_selection库的SelectFromModel类结合带L1以及L2惩罚项的逻辑回归模型，来选择特征的代码如下：              
from sklearn.feature_selection import SelectFromModel           
SelectFromModel(LR(threshold=0.5, C=0.1)).fit_transform(iris.data, iris.target)    

3）基于树模型的特征选择法：树模型中GBDT也可用来作为基模型进行特征选择，使用feature_selection库的SelectFromModel类结合GBDT模型，来选择特征的代码如下：                 
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier               
SelectFromModel(GradientBoostingClassifier()).fit_transform(iris.data, iris.target)              

#### 4.网格搜索交叉验证的作用是什么，并简述网格搜索交叉验证是如何运行的？ 

1）交叉验证是评估方法，将数据集D划分为k个大小相似的互斥子集每个子集Di都尽可能保持数据分布的一致性，即从D中通过分层采样所得。训练时，每次用k-1个子集的并集作为训练集，余下的一个子集作为测试集；如此，可获得k组训练集和测试集，从而进行k次训练和测试，最终返回k次测试结果的均值。     

2）网格搜索法是参数选择的方法，通过穷举法选择出一系列的参数组。它将各个参数可能的取值进行排列组合，列出所有可能的组合结果生成“网格”。    

3）网格搜索加交叉验证就构成了一个不错的调参方式，首先由网格搜索选出各种参数组合，然后将各组合用于模型训练，并使用交叉验证对表现进行评估。在拟合函数尝试了所有的参数组合后，返回一个合适的分类器，自动调整至最佳参数组合。

#### 5.工业界上模型整合有三大类方式？试简述每类方式其思想？（面试题）

1.集体智慧(投票器/BAGGING/随机森林（样本特征随机）。           
这一类方式的思想是每个模型的学习原理是不同的，集合各种模型的学习结果在一起，最终的学习效果也应该会更好些，是并行的算法。        
1）投票器是让融合的各个模型投标表决从而控制过拟合。          
2）BAGGING会对数据做采样从而其中每个模型看到的数据都是不一样的。进而使各个模型的差异化增大。              
3）随机森林比bagging更随机更多样化，更能缓解过拟合。 

2.站在巨人的肩膀看得更远STACKING和BLENDING（用上层模型输出 做为线性模型的特征）       
这类方式是分两层的结构。      
1）Stacking会在上一层模型输出的结果的基础上再去学习一个模型。用上一层estimator的结果作为下一层的特征。     
2）Blending是一个弱化版的stacking它第二层的模型是线性的。      

3.一万小时定律BOOSTING（特点：反复训练，分配不同权重，简单模型叠加adaboost，串行运行）             
1）adaboost 串行运行的，每次运行调整学的不好的部分。      
2）boosting 串行运行的，用比较弱的模型通过多次迭代处理上一轮学习的不好的数据来调整权重，最终达到比较不错的效果。

#### 6.  我们可以将xgboost的中众多参数分类为哪三类？请写出哪些参数可以用什么方式用来控制过拟合？

1.General Parameters(通用参数)：用来设置XGboost的宏观功能。 包括booster，silent，nthread等

2.Boosterparameters：基于模型的参数。     
max_depth,min_child_weight和gamma都是通过控制模型的复杂度来控制过拟合的。   
&emsp;1）gamma参数：表示在节点分裂时，只有分裂后损失函数的值下降了，才会分裂这个节点。Gamma指定了节点分裂所需的最小损失函数下降值。这个参数的值越大，算法越保守。同时也就一定程度上抵抗了过拟合。   
&emsp;2）max_depth：树深太深的话模型就过拟合了，所以通过设置树的深度也可以控制过拟合。 

subsample,colsample_bytree是通过增加随机性来使训练更能抵抗噪声增强健壮性来控制过拟合的。    
&emsp;1） subsample:通过随机抽取部分样本来进行训练可以防止过拟合。   
&emsp;2） eta：降低学习率也是控制过拟合的一种方法，但是要注意增加num_round（学习轮数）。

3.task parameters学习任务参数：这个参数用来控制理想的优化目标和每一步结果的度量方法。     
objective：代价函数。


## 实验题(共2题，每题25分，共计50分)

### 1. 使用XGBoost内置方式，导入iris数据完成分类问题（要求以不同参数设置xgboost运行并比对），最后给出实验总结

In [2]:
from sklearn.datasets import load_iris   
from sklearn.model_selection import train_test_split  
import xgboost as xgb  
from matplotlib import pyplot as plt
from xgboost import plot_importance

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)  
                                                              
dtrain = xgb.DMatrix(X_train, y_train)  
dtest = xgb.DMatrix(X_test, y_test)  


def xgboost(params,num_rounds,dtrain,dtest):  
    watch_list = [(dtrain, 'train'),(dtest, 'eval')]  
    model = xgb.train(params, dtrain, num_rounds,watch_list)  

    pred = model.predict(dtrain)  
    labels = dtrain.get_label()
    e=0
    for index in range(len(pred)):
        if int(pred[index]==labels[index]):
            e += 1        
    print('训练集准确率等于',e/len(pred))

    pred = model.predict(dtest)  
    labels = dtest.get_label()
    e=0
    for index in range(len(pred)):
        if int(pred[index]==labels[index]):
            e += 1        
    print('测试集准确率等于',e/len(pred))
##参数    
params={  
    'booster':'gbtree',  
    'silent': 1,                           # 设置成1则没有运行信息输出，最好是设置为0.  
    'eta': 0.6,                           # 如同学习率(等价于learn_rate)  
    'min_child_weight':3,                  # 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言，假设 h 在 0.01 附  
                                           # 近，min_child_weight 为 1 意味着叶子节点中最少需要包含100个样本。   
                                           # 这个参数控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。  
    'max_depth':3,                         # 构建树的深度，越大越容易过拟合  
    'gamma':0.1,                           # 树的叶子节点上作进一步分区所需的最小损失减少,越大越保守  
    'subsample':0.7,                       # 随机采样训练样本  
    'colsample_bytree':0.7,                # 生成树时进行的列采样   
    'lambda':2,                            # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。  
    'objective':'multi:softmax',
    'num_class':3,
#     'seed':1000

}  
num_rounds = 3                     # 迭代次数  
xgboost(params,num_rounds,dtrain,dtest)

[0]	train-merror:0.038095	eval-merror:0.088889
[1]	train-merror:0.038095	eval-merror:0.088889
[2]	train-merror:0.038095	eval-merror:0.088889
训练集准确率等于 0.9619047619047619
测试集准确率等于 0.9111111111111111


调参与分析：
(有一句mmp我一定要讲，这数据集太小了，动不动就过拟合了。)

1.首先来看 max_depth 对模型的影响：         
原例子中max_depth为3，现在设置max_depth为1

In [3]:
p1 = { 'booster':'gbtree','silent': 1,'eta': 0.6, 'min_child_weight':3, 'max_depth':1,'gamma':0.1, 'subsample':0.7,\
    'colsample_bytree':0.7,  'lambda':2, 'objective':'multi:softmax','num_class':3,}  
num_rounds = 3   
xgboost(p1,num_rounds,dtrain,dtest)

[0]	train-merror:0.038095	eval-merror:0.088889
[1]	train-merror:0.047619	eval-merror:0.066667
[2]	train-merror:0.047619	eval-merror:0.066667
训练集准确率等于 0.9523809523809523
测试集准确率等于 0.9333333333333333


由上可见虽然训练集准确率下降了但是测试集的准确率却上升了，这说明树的深度由3变为了1以后对训练集的描述泛化了，有效的减少了过拟合。

2 然后再看gamma对模型的影响        
Gamma指定了节点分裂所需的最小损失函数下降值。现在保持'max_depth'依然为1,现在设置gamma为10看看

In [4]:
p2 = { 'booster':'gbtree','silent': 1,'eta': 0.6, 'min_child_weight':3, 'max_depth':1,'gamma':10, 'subsample':0.7,\
    'colsample_bytree':0.7,  'lambda':2, 'objective':'multi:softmax','num_class':3,}  
num_rounds = 3 
xgboost(p2,num_rounds,dtrain,dtest)

[0]	train-merror:0.038095	eval-merror:0.088889
[1]	train-merror:0.047619	eval-merror:0.044444
[2]	train-merror:0.047619	eval-merror:0.044444
训练集准确率等于 0.9523809523809523
测试集准确率等于 0.9555555555555556


由上可见虽然训练集准确率不变的情况下测试集的准确率上升了，gamma越大节点越难分裂，算法越保守。同时也就一定程度上抵抗了过拟合。

3.接下来来调整subsample参数          
分别设置subsample为0.9和0.2

In [5]:
p3 = { 'booster':'gbtree','silent': 1,'eta': 0.6, 'min_child_weight':3, 'max_depth':1,'gamma':10, 'subsample':0.9,\
    'colsample_bytree':0.7,  'lambda':2, 'objective':'multi:softmax','num_class':3,}  
num_rounds = 3 
xgboost(p3,num_rounds,dtrain,dtest)

[0]	train-merror:0.038095	eval-merror:0.088889
[1]	train-merror:0.047619	eval-merror:0.044444
[2]	train-merror:0.047619	eval-merror:0.044444
训练集准确率等于 0.9523809523809523
测试集准确率等于 0.9555555555555556


In [6]:
p3 = { 'booster':'gbtree','silent': 1,'eta': 0.6, 'min_child_weight':3, 'max_depth':1,'gamma':10, 'subsample':0.2,\
    'colsample_bytree':0.7,  'lambda':2, 'objective':'multi:softmax','num_class':3,}  
num_rounds = 3 
xgboost(p3,num_rounds,dtrain,dtest)

[0]	train-merror:0.628571	eval-merror:0.755556
[1]	train-merror:0.628571	eval-merror:0.755556
[2]	train-merror:0.628571	eval-merror:0.755556
训练集准确率等于 0.37142857142857144
测试集准确率等于 0.24444444444444444


分别设置subsample为0.2和0.9时，模型分别为过拟合与欠拟合的状态。

4.经过测试colsample_bytree 参数也可达到与subsample类似的效果。

In [7]:
p4 = { 'booster':'gbtree','silent': 1,'eta': 0.6, 'min_child_weight':3, 'max_depth':1,'gamma':10, 'subsample':0.9,\
    'colsample_bytree':0.2,  'lambda':2, 'objective':'multi:softmax','num_class':3,}  
num_rounds = 3 
xgboost(p4,num_rounds,dtrain,dtest)

[0]	train-merror:0.228571	eval-merror:0.311111
[1]	train-merror:0.228571	eval-merror:0.311111
[2]	train-merror:0.228571	eval-merror:0.311111
训练集准确率等于 0.7714285714285715
测试集准确率等于 0.6888888888888889


In [8]:
p4 = { 'booster':'gbtree','silent': 1,'eta': 0.6, 'min_child_weight':3, 'max_depth':1,'gamma':10, 'subsample':0.9,\
    'colsample_bytree':0.7,  'lambda':2, 'objective':'multi:softmax','num_class':3,}  
num_rounds = 3 
xgboost(p4,num_rounds,dtrain,dtest)

[0]	train-merror:0.038095	eval-merror:0.088889
[1]	train-merror:0.047619	eval-merror:0.044444
[2]	train-merror:0.047619	eval-merror:0.044444
训练集准确率等于 0.9523809523809523
测试集准确率等于 0.9555555555555556


5.最后调整学习率
初始学习率为0.6，现在分别设置为0.01和10：

In [12]:
p5 = { 'booster':'gbtree','silent': 1,'eta': 0.001, 'min_child_weight':3, 'max_depth':1,'gamma':10, 'subsample':0.7,\
    'colsample_bytree':0.7,  'lambda':2, 'objective':'multi:softmax','num_class':3,}  
num_rounds = 2 
xgboost(p5,num_rounds,dtrain,dtest)

[0]	train-merror:0.038095	eval-merror:0.088889
[1]	train-merror:0.047619	eval-merror:0.044444
训练集准确率等于 0.9523809523809523
测试集准确率等于 0.9555555555555556


In [11]:
p6= { 'booster':'gbtree','silent': 1,'eta': 10, 'min_child_weight':3, 'max_depth':1,'gamma':10, 'subsample':0.7,\
    'colsample_bytree':0.7,  'lambda':2, 'objective':'multi:softmax','num_class':3,}  
num_rounds = 2 
xgboost(p5,num_rounds,dtrain,dtest)

[0]	train-merror:0.038095	eval-merror:0.088889
[1]	train-merror:0.047619	eval-merror:0.044444
训练集准确率等于 0.9523809523809523
测试集准确率等于 0.9555555555555556


在迭代次数都为2的时候学习率为10的过拟合了，下面维持学习率不变，增加迭代次数num_rounds为30

In [13]:
num_rounds=30
xgboost(p5,num_rounds,dtrain,dtest)

[0]	train-merror:0.038095	eval-merror:0.088889
[1]	train-merror:0.047619	eval-merror:0.044444
[2]	train-merror:0.047619	eval-merror:0.044444
[3]	train-merror:0.047619	eval-merror:0.044444
[4]	train-merror:0.047619	eval-merror:0.044444
[5]	train-merror:0.047619	eval-merror:0.044444
[6]	train-merror:0.047619	eval-merror:0.044444
[7]	train-merror:0.047619	eval-merror:0.044444
[8]	train-merror:0.047619	eval-merror:0.044444
[9]	train-merror:0.047619	eval-merror:0.044444
[10]	train-merror:0.047619	eval-merror:0.044444
[11]	train-merror:0.047619	eval-merror:0.044444
[12]	train-merror:0.047619	eval-merror:0.044444
[13]	train-merror:0.047619	eval-merror:0.044444
[14]	train-merror:0.047619	eval-merror:0.044444
[15]	train-merror:0.047619	eval-merror:0.044444
[16]	train-merror:0.047619	eval-merror:0.044444
[17]	train-merror:0.047619	eval-merror:0.044444
[18]	train-merror:0.047619	eval-merror:0.044444
[19]	train-merror:0.047619	eval-merror:0.044444
[20]	train-merror:0.038095	eval-merror:0.044444
[2

测试集准确率提升了由上可见学习率减小的同时增加迭代次数的话，也可以提高模型表现效果。

### 2. 使用XGBoost的sklearn接口，对KaggleCredit2数据完成信用卡欺诈项目的建模及分析（要求以不同参数设置xgboost运行并比对），最后给出实验总结

- KaggleCredit2数据文件 位于/home/sxy-s3/0.Teacher/Exam/KaggleCredit2.csv.zip，请勿复制或移动该文件

In [14]:
from sklearn.model_selection import train_test_split      
from sklearn import metrics  
from xgboost.sklearn import XGBClassifier  
import pandas as pd
pd.set_option('display.max_columns', 500)
import zipfile
with zipfile.ZipFile('/home/sxy-s3/0.Teacher/Exam/KaggleCredit2.csv.zip', 'r') as z:
    f = z.open('KaggleCredit2.csv')
    data = pd.read_csv(f, index_col=0)
y = data['SeriousDlqin2yrs']
X = data.drop('SeriousDlqin2yrs', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)  

In [15]:
clf = XGBClassifier(  
silent=0 ,                                      #设置成1则没有运行信息输出，最好是设置为0.是否在运  
                                                #行升级时打印消息。  
#nthread=4,                                     # cpu 线程数 默认最大  
learning_rate= 0.01,                             # 如同学习率(eta)  
min_child_weight=1,   
                                                # 这个参数默认是 1，是每个叶子里面 h 的和至少是多  
                                                # 少，对正负样本不均衡时的 0-1 分类而言  
                                                #，假设 h 在 0.01 附近，min_child_weight 为 1 意味  
                                                # 着叶子节点中最少需要包含 100 个样本。  
                                                # 这个参数非常影响结果，控制叶子节点中二阶导的和的  
                                                # 最小值，该参数值越小，越容易 overfitting。  
max_depth=6,                                    # 构建树的深度，越大越容易过拟合  
gamma=0,                                        # 树的叶子节点上作进一步分区所需的最小损失减少,越大  
                                                # 越保守，一般0.1、0.2这样子。  
subsample=1,                                    # 随机采样训练样本 训练实例的子采样比  
max_delta_step=0,                               # 最大增量步长，我们允许每个树的权重估计。  
colsample_bytree=1,                             # 生成树时进行的列采样   
reg_lambda=1,                                   # 控制模型复杂度的权重值的L2正则化项参数，参数越  
                                                # 大，模型越不容易过拟合。  
#reg_alpha=0,                                   # L1 正则项参数  
#scale_pos_weight=1,                            # 如果取值大于0的话，在类别样本不平衡的情况下有助于     
                                                # 快速收敛，平衡正负权重  
#objective= 'multi:softmax',                    # 多分类的问题 指定学习任务和相应的学习目标  
#num_class=10,                                  # 类别数，多分类与 multisoftmax 并用  
n_estimators=100,                               # 树的个数  
seed=1000                                       # 随机种子  
#eval_metric= 'auc'  
)  
clf.fit(X_train,y_train,eval_metric='auc')  
y_true, y_pred = y_test, clf.predict(X_test)  
print("Accuracy : %.4g" % metrics.accuracy_score(y_true, y_pred)) #显示精度，越大越好  
  

Accuracy : 0.9346


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


下面用网格搜索来进行调参：     
1.首先对 max_depth 进行设置分别为1,2,5,10

In [17]:
from sklearn.model_selection import GridSearchCV
model = XGBClassifier() 
param_grid= {'max_depth':[1,2,5,10]}
grid_search = GridSearchCV(estimator = XGBClassifier(silent=0,max_depth=6,gamma=0,n_estimators=100,subsample=1,reg_lambda=1,seed=10), param_grid=param_grid, scoring='roc_auc')
grid_result = grid_search.fit(X_train, y_train)

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

0.849481 (0.001344) with: {'max_depth': 1}
0.854458 (0.000561) with: {'max_depth': 2}
0.854802 (0.000341) with: {'max_depth': 5}
0.841760 (0.001650) with: {'max_depth': 10}
Best: 0.854802 using {'max_depth': 5}


如上所示树深度选5的效果是最好的，大了会过拟合，小了则为欠拟合。    
2.下面进行gamma参数调优Gamma参数取值范围可以很大，我这里取值范围设置为6，10，15.（不敢设太多怕卡）

In [18]:
from sklearn.model_selection import GridSearchCV
model = XGBClassifier() 
param_grid= {'gamma':[6, 10, 15]}
grid_search = GridSearchCV(estimator = XGBClassifier(silent=0,max_depth=6,gamma=0,n_estimators=100,subsample=1,reg_lambda=1,seed=1000), param_grid=param_grid, scoring='roc_auc')
grid_result = grid_search.fit(X_train, y_train)

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

0.855295 (0.000656) with: {'gamma': 6}
0.855476 (0.000593) with: {'gamma': 10}
0.854653 (0.000437) with: {'gamma': 15}
Best: 0.855476 using {'gamma': 10}


如上所示gamma为10的得分最高。         
3.下面subsample 和 colsample_bytree 俩参数一起调

In [19]:
from sklearn.model_selection import GridSearchCV
model = XGBClassifier() 
param_grid= {'subsample':[0.3, 0.6, 0.9], 'colsample_bytree':[0.3, 0.6, 0.9]}
grid_search = GridSearchCV(estimator = XGBClassifier(silent=0,max_depth=6,gamma=0,n_estimators=100,subsample=1,reg_lambda=1,seed=1000), param_grid=param_grid, scoring='roc_auc')
grid_result = grid_search.fit(X_train, y_train)

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


0.850987 (0.001560) with: {'subsample': 0.3, 'colsample_bytree': 0.3}
0.853690 (0.001188) with: {'subsample': 0.6, 'colsample_bytree': 0.3}
0.854334 (0.001176) with: {'subsample': 0.9, 'colsample_bytree': 0.3}
0.851424 (0.000463) with: {'subsample': 0.3, 'colsample_bytree': 0.6}
0.853037 (0.000586) with: {'subsample': 0.6, 'colsample_bytree': 0.6}
0.854497 (0.000496) with: {'subsample': 0.9, 'colsample_bytree': 0.6}
0.851143 (0.000570) with: {'subsample': 0.3, 'colsample_bytree': 0.9}
0.852407 (0.000987) with: {'subsample': 0.6, 'colsample_bytree': 0.9}
0.853312 (0.000522) with: {'subsample': 0.9, 'colsample_bytree': 0.9}
Best: 0.854497 using {'subsample': 0.9, 'colsample_bytree': 0.6}


最佳选择为'colsample_bytree'= 0.6与 'subsample'= 0.9            
4.最后依然是调学习率  

In [20]:
from sklearn.model_selection import GridSearchCV
model = XGBClassifier() 
param_grid= {'learning_rate':[0.01, 0.1, 1, 10]}
grid_search = GridSearchCV(estimator = XGBClassifier(silent=0,max_depth=6,gamma=0,n_estimators=100,subsample=1,reg_lambda=1,seed=1000), param_grid=param_grid, scoring='roc_auc')
grid_result = grid_search.fit(X_train, y_train)

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

0.845564 (0.004012) with: {'learning_rate': 0.01}
0.853172 (0.000652) with: {'learning_rate': 0.1}
0.787670 (0.002530) with: {'learning_rate': 1}
0.556187 (0.014270) with: {'learning_rate': 10}
Best: 0.853172 using {'learning_rate': 0.1}


最佳选择为0.1    
综上所得，在用xboost模型时合理调整参数的取值可以在很大程度上通过使模型避免过拟合与欠拟合的方式，改善模型对数据集的表示效果。sklearn和xgboost本身也都提供了很多相应的调参和评估方法。

### 本阶段课程意见反馈题(非必答，不送分)
#### 请同学围绕以下两点进行回答：
- 自身总结：请您对您自己在本周课程的学习，收获，技能掌握等方面进行一次总结 ，也包括有哪些方面存在哪些不足，欠缺，困惑。作为将来回顾学习路径时的依据。


- 课程反馈：请就知识点，进度，难易度，教学方式，考试方式及难易度等方面向我们反馈，督促我们进行更有效的改进。

<hr>
<center><h1>####答卷结束####</h1></center>